Source [link](https://www.geeksforgeeks.org/python/decorators-in-python/)

**Decorators** are:
1. a flexible way to modify or extend behavior of functions or methods, without changing their actual code.
2. essentially a function that takes another function as an argument and returns a new function with enhanced functionality.

**Note**: it's important to remember that in Python functions are objects. This means they behave like any other object (such as integers, strings, or lists): they can be assigned to variables, passed as arguments, returned from other functions, and stored in data structures.

In [110]:
def dec(func):
    def wrapper(*args, **kwargs):
        print("Before function call")
        result = func(*args, **kwargs)
        print("After function call")
        return result
    return wrapper

print(type(dec)) # the dec is an instance of the class 'function'

<class 'function'>


In [70]:
def f1(name):
    print(f"Hello, {name}!")

@dec
def f2(name):
    print(f"Hello, {name}!")

In [71]:
f1("X")

Hello, X!


In [72]:
f2("Y")

Before function call
Hello, Y!
After function call


`@dec` syntax is a shorthand for `f2 = dec(f2)`.

# Two Main Types of Decorators:

## 1. Function Decorators

Used to wrap and enhance functions by adding extra behavior before or/and after the original function runs.

In [76]:
def func_dec(func):
    def wrapper(*args, **kwargs):
        print("I'm a function decorator!")
        return func(*args, **kwargs)
    return wrapper

@func_dec
def func(*args, **kwargs):
    print(f"Arguments: {args}, Keyword Arguments: {kwargs}")

func(1, 2, 3, a=4, b=5)

I'm a function decorator!
Arguments: (1, 2, 3), Keyword Arguments: {'a': 4, 'b': 5}


## 2. Method Decorators

Special decorators used for methods inside a class.

In [77]:
def method_dec(method):
    def wrapper(self, *args, **kwargs):
        print("I'm a method decorator!")
        return method(self, *args, **kwargs)
    return wrapper

class MyClass1:
    @method_dec
    def my_method(self, x):
        print(f"My method called with argument: {x}")

obj = MyClass1()
obj.my_method(10)

I'm a method decorator!
My method called with argument: 10


In [78]:
MyClass1.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 7,
              'my_method': <function __main__.method_dec.<locals>.wrapper(self, *args, **kwargs)>,
              '__static_attributes__': (),
              '__dict__': <attribute '__dict__' of 'MyClass1' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass1' objects>,
              '__doc__': None})

# Built-in Decorators



## 1. @classmethod

It is used to define a method that operates on class itself (i.e., it uses cls). Class methods can access and modify class state that applies across all instances of class.

In [84]:
class MyClass2:
    y = 0

    @classmethod
    def method_1(cls, y):
        cls.y = y
    
    def method_2(self, y):
        self.y = y

obj = MyClass2()

In [85]:
obj.__dict__ # Should be empty initially as 'y' is a class attribute

{}

In [86]:
obj.method_2(5)
print(MyClass2.y) # Should still be 0

0


In [87]:
obj.__dict__ # Now should have a 'y' attribute with value 5

{'y': 5}

In [88]:
obj.method_1(15)
print(MyClass2.y) # Now should be 15

15


In [89]:
MyClass2.method_1(25) # Using class method to set y to 25
print(MyClass2.y)

25


In [ ]:
MyClass2.method_2(0)

# This raises an error because method_2 is not a class method.
# It expects an instance of MyClass2 as its first argument, but
# when called on the class itself, the interpreter passes only the
# provided argument (0), resulting in a mismatch.

TypeError: MyClass2.method_2() missing 1 required positional argument: 'y'

## 2. @staticmethod

It is used to define a method that does not depend on either the class itself or any specific instance of that class.

In [104]:
class MyClass3:

    @staticmethod
    def greet(name): # Static method does not take self or cls as first argument
        print(f"Hello, {name}!")

obj = MyClass3()

In [105]:
obj.greet("Alice")

Hello, Alice!


In [95]:
MyClass3.greet("Alice")

Hello, Alice!


## 3. @property

It is used to define getters and setters for class attributes keeping the interface of a "normal attribute". It's often used in combination with a private attibute.

In [107]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self): # This acts as a getter for radius
        return self._radius

    @radius.setter
    def radius(self, value): # This acts as a setter for radius
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius cannot be negative")

    @property
    def area(self):
        return 3.14159 * (self._radius ** 2)

# Using the property
c = Circle(5)
print(c.radius) 
print(c.area)    
c.radius = 10
print(c.area)

5
78.53975
314.159


## 3. @lru_cache

It is a decorator from Python’s `functools` module that memoizes function calls by storing the results of recent executions in a dict keyed by the hashed function’s arguments. The cache follows a Least Recently Used (LRU) eviction policy, meaning that when it reaches its `maxsize`, the least recently accessed entries are discarded.

A function can benefit from `@lru_cache` if:
1. Same inputs → same outputs (pure or mostly pure)
2. Arguments are hashable
3. The function is called multiple times with repeated arguments

It can save time when an expensive or I/O bound function is periodically called with the same arguments.

The cache is threadsafe so that the wrapped function can be used in multiple threads. This means that the underlying data structure will remain coherent during concurrent updates.

**Note:** The internal cache persists after the function call returns.  
This is because Python functions are **heap-allocated objects**. The cache is stored as part of the **wrapped function’s state** (attributes / closure). When `f(x)` finishes executing:
1. The call’s **stack frame is destroyed**,
2. The **function object remains alive**, and
3. The **cache dictionary remains alive with it**.

In [1]:
from functools import lru_cache
from time import time

In [2]:
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

def fib_timer(fib, n):
    start = time()
    result = fib(n)
    end = time()
    print(f"Fibonacci({n}) = {result}, computed in {end - start:.6f} seconds")

In [3]:
fib_timer(fibonacci, 40)  # This will take a noticeable amount of time

Fibonacci(40) = 102334155, computed in 14.084722 seconds


In [ ]:
@lru_cache(maxsize=None)
def fibonacci_cached(n):
    if n <= 1:
        return n
    return fibonacci_cached(n-1) + fibonacci_cached(n-2)

In [ ]:
fib_timer(fibonacci_cached, 40)  # This should be much faster

Fibonacci(40) = 102334155, computed in 0.000037 seconds


In [ ]:
fibonacci_cached.cache_info()

CacheInfo(hits=38, misses=41, maxsize=None, currsize=41)

**Problem**(!):

In [ ]:
import random

def not_pure_function(x):
    return x + random.randint(1, 10)

not_pure_function(5), not_pure_function(5), not_pure_function(5), not_pure_function(5)

(13, 11)

In [10]:
@lru_cache(maxsize=None)
def not_pure_function_cached(x):
    return x + random.randint(1, 10)

not_pure_function_cached(5), not_pure_function_cached(5), not_pure_function_cached(5), not_pure_function_cached(5)

(14, 14, 14, 14)

Although the decorator is extremely useful, it should not be used in all situations. The code above highlights a problem: the function should return a random number for each call, but since the input parameter is always the same, the result is cached after the first invocation. As a consequence, subsequent calls return the same value instead of generating a new random number, defeating the intended behavior of the function.